# Lab 2: Flexure of the oceanic lithosphere under island chains 

In this lab, we will solve the plate flexure equation for an ocean island load numerically using finite-differences. We will compare the model results to read data from Hawai'i and estimate the effective elastic thickness of the Pacific plate underneath the islands.

Learning objectives:

* Build on the theorical knowledge acquired in [Lecture 2](https://www.leouieda.com/envs398/slides/2-isostasy/).
* Learn how to apply the finite-difference method to solve partial differential equations.
* Gain a practical understanding of plate flexure.


## General instructions

This is a [Jupyter notebook](https://jupyter.org/) running in [Jupyter Lab](https://jupyterlab.readthedocs.io/en/stable/). The notebook is a programming environment that mixes code (the parts with `[1]: ` or similar next to them) and formatted text/images/equations with [Markdown](https://www.markdownguide.org/basic-syntax) (like this part right here).

Quick start guide:

* **Edit** any cell (blocks of code or text) by double clicking on it.
* **Execute** a code or Markdown cell by typing `Shift + Enter` after selecting it.
* The current active cell is the one with a **blue bar next to it**.
* You can run cells **in any order** as long as the code sequence makes sense (it's best to go top-to-bottom, though).
* To copy any file to the current directory, drag and drop it to the file browser on the left side.
* Notebook files have the extension `.ipynb`.

## The plate flexure equation

The equation describing the flexure ($w$) of an elastic plate subject to a vertical load is:

$ D\dfrac{\partial^4 w}{\partial x^4} = q(x) - (\rho_m - \rho_w) g w $

in which $w$ is the flexure (how much the plate bends up or down), $x$ is the position, $D$ is the flexural rigidity of the plate, $rho_c$ and $rho_m$ are the crust and mantle densities, $g$ is gravity (for these purposes, assume $9.8\ m/s^2$), and $q(x)$ is a force density applied by the load on the plate (the weight of the topography). The term $(\rho_m - \rho_w) g w$ is the restoring hydrostatic force (buoyancy) from replacing mantle material at depth with water on top because of the flexure.

One of the the assumptions we're taking here is that the lithosphere has a density $\rho_m$ same as the underlying mantle. So we're effectively disregarding the crustal part of the lithosphere. 

The flexural rigidity can be described in terms of the plate's elastic parameters:

$D = \dfrac{E T_e^3}{12(1 - \nu^2)}$

$E$ is [Young's modulus](https://en.wikipedia.org/wiki/Young's_modulus), $\nu$ is [Poisson's ratio](https://en.wikipedia.org/wiki/Poisson's_ratio), and $T_e$ is the *effective elastic thickness* (the reference books use $h$ for this but $T_e$ is very common in the literature).

The **load force per unit area** $q(x)$ is the weight of the load applied on the lithosphere. For a load of height $h$ and density $\rho_c$, the weight per unit area is (negative because the force is applied downwards):

$q(x) = -g \rho_c h(x)$

**Note:** For a detailed derivation of these equations and analytical solutions, see "Chapter 3: Elasticity and Flexure" of "Geodynamics" by Turcotte and Schubert.

## Boundary conditions

We need to consider the appropriate boundary conditions for the loading of the oceanic lithosphere by an island chain. If we assume that the load is applied at the middle of the plate, we can consider the boundaries as stable lithosphere under no load. In that case, we don't expect to see any flexure at the boundaries:

$w(x=0) = 0$ and $w(x=x_{max}) = 0$

We also need to imposed conditions on the derivative of $w$. If there is no flexure, we can also assume that the first derivative of flexure is zero (the plate approaches the boundaries horizontally, not at an angle):

$\dfrac{\partial w}{\partial x}(x=0) = 0$ and $\dfrac{\partial w}{\partial x}(x=x_{max}) = 0$

## Finite-difference solution

The finite-difference method is a way of calculating an approximate solution to partial differential equations. Numerical solutions allows us to solve problems for complex sources and mediums for which analytical solutions might not have been found.

Remembering our calculus, the derivative is defined as:

$\dfrac{\partial w}{\partial x}(x) = \lim_{\Delta x \to 0} \dfrac{w(x + \Delta x) - w(x)}{\Delta x}$

The main concept for finite-differences is approximating the derivative by eliminating the limit above and assuming that $\Delta x$ is sufficiently small:

$\dfrac{\partial w}{\partial x}(x) \approx \dfrac{w(x + \Delta x) - w(x)}{\Delta x}$

The numerical process involves discretizing the continuous value $w(x)$ into evenly spaced points $w_0, w_1, \ldots, w_{N-1}$ with $\Delta x$ spacing between them. In that case, the first derivative between points $w_0$ and $w_1$ can be approximated by:

$\dfrac{\partial w}{\partial x}(x_0) \approx \dfrac{w_1 - w_0}{\Delta x}$

Propagating this to other higher order derivatives: 

![Finite-differences for up to fourth order derivatives.](https://github.com/leouieda/envs398/raw/master/labs/figures/finite-differences-grid.png)

With finite-differences, we can calculate the derivative at $x_2$ from values of $w$ in the 5 points around $x_2$.

Using this formulation, the flexure equation in finite-differences form is:

$ 
D \dfrac{w_{i+2} - 4w_{i+1} + 6w_i - 4w_{i-1} + w_{i-2}}{\Delta x^4} = q_i - (\rho_m - \rho_w) g w_i
$

Here, we're using $w_i$ to represent the flexure at an arbitrary node in the grid. Notice that we have discretized the load $q(x)$ at the same locations as well to get $q_i$.

The **boundary conditions** also need to be converted to discrete finite-differences:

$w_0 = 0$ and $w_{N-1} = 0$

$\dfrac{w_1- w_0}{\Delta x} = 0$ and $\dfrac{w_{N-1} - w_{N-2}}{\Delta x} = 0$.

## Solving for $w$

**The flexure $w$ is what we want to calculate so they are the unknowns in this equation.** So we can re-arange the equation above to isolate the $w$ on the left-hand side of the equation and any term not containing $w$ on the other. Also eliminate the division to make our lives easier. 

$ 
D w_{i+2} - 4 D w_{i+1} + \left[6D + \Delta x^4 (\rho_m - \rho_w) g\right] w_i - 4 D w_{i-1} + D w_{i-2} = \Delta x^4 q_i
$

Since we have values of $w$ for multiple values of $i$, we can form an equation system (with the boundary conditions as well):

$
\begin{align*}
  D w_{4} - 4 D w_{3} + \left[6D + \Delta x^4 (\rho_m - \rho_w) g\right] w_2 - 4 D w_{1} + D w_{0} &= \Delta x^4 q_2  \\ 
  D w_{5} - 4 D w_{4} + \left[6D + \Delta x^4 (\rho_m - \rho_w) g\right] w_3 - 4 D w_{2} + D w_{1} &= \Delta x^4 q_3  \\ 
  \vdots & \\
  D w_{i+4} - 4 D w_{i+3} + \left[6D + \Delta x^4 (\rho_m - \rho_w) g\right] w_{i+2} - 4 D w_{i+1} + D w_{i} &= \Delta x^4 q_{i+2}  \\ 
  \vdots & \\
  D w_{N-1} - 4 D w_{N-2} + \left[6D + \Delta x^4 (\rho_m - \rho_w) g\right] w_{N-3} - 4 D w_{N-4} + D w_{N-5} &= \Delta x^4 q_{N-3} \\
  w_0 &= 0  \\
  w_{N-1} &= 0  \\
  w_1 - w_0 &= 0  \\
  w_{N-1} - w_{N-2} &= 0 
\end{align*}
$

With this system, we have $N$ unknowns and $N$ equations ($N-4$ for the flexure equation and $4$ for the boundary conditions). **Solving the system will give us the flexure ($w$) for a given load $q$.**

Another way of writing this equation system is using a matrix notation:

$
\begin{bmatrix}
  D  & -4D & \left[6D + \Delta x^4 (\rho_m - \rho_w) g\right] & -4D & D & 0 & 0 & \ldots & 0 & 0 \\ 
  0 & D  & -4D & \left[6D + \Delta x^4 (\rho_m - \rho_w) g\right] & -4D & D & 0 & \ldots & 0 & 0 \\ 
  \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots  \\
  1 & 0 & 0 & 0 & 0 & 0 & 0 & \ldots & 0 & 0  \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & \ldots & 0 & 1  \\
  -1 & 1 & 0 & 0 & 0 & 0 & 0 & \ldots & 0 & 0  \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & \ldots & -1 & 1
 \end{bmatrix}
\begin{bmatrix}
  w_0 \\
  w_1 \\
  w_2 \\
  w_3 \\
  w_4 \\
  w_5 \\
  w_6 \\
  \vdots \\
  w_{N-1} 
 \end{bmatrix}
 =
\begin{bmatrix}
  \Delta x^4 q_2 \\
  \Delta x^4 q_3 \\
  \vdots \\
  0\\
  0\\
  0\\
  0
 \end{bmatrix}
$

$\bar{\bar{A}}\bar{w} = \bar{b}$

Calculating $w$ is now a matter of solving the linear system using matrix $\bar{\bar{A}}$ and the right-hand-side vector $\bar{b}$.

## Import things

As before, the first thing to do is load the Python libraries that we'll be using. We'll group all our imports here at the top to make it easier to see what we're using.

In [ ]:
# The base of the entire scientific Python stack
import numpy as np
# For making plots and figures
import matplotlib.pyplot as plt
# To load and operate on data tables
import pandas as pd
# Scipy defines a bunch of scientific goodness on top of numpy
# scipy.optimize is the module for function minimization
import scipy.optimize

## Setup a simplified model first

To test our implementation, let's try setting up a model with a load in a single point in the middle of the grid. First, we have to make the grid (array of $x$ values and associated $\Delta x$).

In [ ]:
number_of_nodes = 500
profile_length = 2500 # km
x = np.linspace(0, profile_length, number_of_nodes)
delta_x = x[1] - x[0]
print(delta_x, "km")

Now define the physical properties of the plate and the surrounding medium:

In [ ]:
gravity = 9.8e-3 # km/s²
density_contrast = 3.3e12 - 1.0e12 # kg/km³
effective_elastic_thickness = 80 # km
young_modulus = 70e12  # kg/(s²km)
poisson_modulus = 0.25  # dimensionless
flexural_rigidity = young_modulus * effective_elastic_thickness**3 / (12 * (1 - poisson_modulus**2))  # kg.km²/s²

Now we can define the load vector $q$. It will have the same number of elements as `x` and we'll place a weight in the middle of it.

In [ ]:
load_height = 5 # km
load_density = 2.8e12 # kg/km³
# Make an array with the same size as x but filled with zeros
load = np.zeros_like(x)
# Use the equation for the force per unit area given above
load[number_of_nodes // 2] = - gravity * load_density * load_height  # kg/(km.s²)

----

### Your turn 

Plot the load vector using matplotlib in the cell below:

In [ ]:
plt.figure()
plt.title("Applied load")
# Complete with your own code below


plt.show()

----

Now that we have a load and physical properties of the medium, let's define the equation system (the $\bar{\bar{A}}$ matrix and $\bar{b}$ vector). Let's start with the right-hand-side of the equation.

In [ ]:
# Start with an empty array of the right size
right_hand_side = np.empty(number_of_nodes)

# Populate the part for the flexure equation. 
# The range function generates numbers between the given range in increments of 1.
# Leave the last 4 for later because they are the boundary conditions.
for i in range(0, number_of_nodes - 4):
    # Taken straight from the equations above
    right_hand_side[i] = delta_x**4 * load[i + 2]
    
# Now add the boundary conditions. 
# Remember that in Python we can index arrays backwards (starting from -1).
right_hand_side[-4 : ] = 0

----

### Your turn

Fill in the code below to create the $\bar{\bar{A}}$ matrix. It will be very similar to the code above.

In [ ]:
# Fill in the shape of the matrix = (number of rows, number of columns)
eq_system = np.zeros((    ))

# Populate the flexure part of the equation
for i in range(0, number_of_nodes - 4):
    
    
# Add the boundary conditions
eq_system[    ] = 1


To check that your matrix is correct, let's estimate the flexure by solving the linear system and compare it with a saved solution that I generated.

In [ ]:
flexure = np.linalg.solve(eq_system, right_hand_side)

Make a plot of the estimated flexure and compare with the saved figure below.

In [ ]:
plt.figure(figsize=(8, 4))
# Fill in with your own code


plt.show()

#### **Reference figure**

![Flexure of a point load.](https://github.com/leouieda/envs398/raw/master/labs/figures/flexure-of-a-point-load.png)

----

## Flexure of Hawai'i

Now we can try to model the flexure of the lithosphere in Hawai'i due to the load of the islands. 

## Load the data

First, let's load the profile data from the file `hawaii-profile.csv` (if downloading the data, put in a `data` folder along side this notebook).

In [ ]:
hawaii = pd.read_csv("data/hawaii-profile.csv")
hawaii

This is a `pandas.DataFrame`. You can access the individual columns of data like so:

In [ ]:
print(hawaii.distance_km)

----

### Your turn

Make a plot of the bathymetry profile in the cell below.

In [ ]:
plt.figure(figsize=(10, 3))
# Fill in with your own code


plt.show()

----

## Remove the thermal swell

We can see from the bathymetry profile that there is a long-wavelength component (almost a gaussian shape) right around the islands. This is due to the thermal swell of the hotspot and not part of the flexure response of the oceanic lithosphere. The swell is isostatically compensanted, as seen in the gravity disturbance (**what mechanism explains this?**). So we have to first remove it.

We'll fit a gaussian curve to the profile data and remove it from our observations to obtain a *residual*.

First, define the gaussian function:

$ 
h_{gauss} (x) = h_{0} + G \exp \left(-\dfrac{(x - x_0)^2}{\sigma^2}\right)
$


In [ ]:
def gaussian(x, h0, G, x0, sigma):
    h_gauss = h0 + G * np.exp(-(x - x0)**2/sigma**2)
    return h_gauss

To estimate the parameters of this function ($h_0$, $G$, $x_0$, $\sigma$), we will eyeball it:

In [ ]:
swell = gaussian(hawaii.distance_km, h0=-5.5, G=1.5, x0=1300, sigma=450)

In [ ]:
plt.figure(figsize=(10, 3))
plt.plot(hawaii.distance_km, hawaii.topography_km, "-k")
plt.plot(hawaii.distance_km, swell, "-r")
plt.xlim(hawaii.distance_km.min(), hawaii.distance_km.max())
plt.xlabel("Distance (km)")
plt.ylabel("Topography (km)")
plt.title("Topography/bathymetry across Oahu and thermal swell")
plt.show()

Now we can calculate a residual topography and store it in our original `pandas.DataFrame`.

In [ ]:
hawaii["residual_topography_km"] = hawaii.topography_km - swell

`hawaii.residual_topography_km` is the data that we're going to try to fit.

In [ ]:
plt.figure(figsize=(10, 3))
plt.plot(hawaii.distance_km, hawaii.residual_topography_km)
plt.xlim(hawaii.distance_km.min(), hawaii.distance_km.max())
plt.xlabel("Distance (km)")
plt.ylabel("Topography (km)")
plt.title("Residual topography across Oahu")
plt.show()

## Estimate the load

Now we can estimate the load as the central part of our profile (the island mass) assuming that it stops at 0.

In [ ]:
# Find the indices corresponding to the island load
island = np.logical_and(hawaii.distance_km > 1100, hawaii.distance_km < 1300)

# Create an array with the topography values of just the load
oahu_height = np.zeros_like(hawaii.distance_km)
oahu_height[island] = hawaii.residual_topography_km[island]

# Create an array with the load weight (same equation as before)
oahu_load = - gravity * load_density * oahu_height  # kg/(km.s²)

---

### Your turn

Plot the load height (`oahu_height`) along side the residual topography.

In [ ]:
plt.figure()
# Fill in with your own code


plt.show()

## Estimate the effective elastic for Hawai'i

Now we can use the code developed earlier to calculate the predicted flexure of the island load. The **parameter we're interested in is the effective elastic thickness**.

----

### Your turn

Calculate the flexure of the island load.

You will have to copy your code from previously and replace the point load with our own. The `x` coordinates are taken from the profile (`hawaii.distance_km`). Fill in the required sections of the function below that does the flexure calculation. This will make it easier for you to vary the parameters and execute the code later on. For example:

In [ ]:
def elastic_flexure(x, load, effective_elastic_thickness):
    # Calculate the physical parameters
    gravity = 9.8e-3 # km/s²
    density_contrast = 3.3e12 - 1.0e12 # kg/km³
    young_modulus = 70e12  # kg/(s²km)
    poisson_modulus = 0.25  # dimensionless
    flexural_rigidity = young_modulus * effective_elastic_thickness**3 / (12 * (1 - poisson_modulus**2))  # kg.km²/s²
    
    # Get the grid spacing
    delta_x = x[1] - x[0]    
    
    # Generate the right-hand side vector    
    # Fill in with your code here
    
    # Generate the equation system matrix
    # Fill in with your code here
    
    # Solve the system for the flexure
    # Fill in with your code here
    
    # Return the calculate flexure out of the function
    return flexure

Now we can use your function to calculate and plot the flexure:

In [ ]:
hawaii_flexure = elastic_flexure(hawaii.distance_km, oahu_load, effective_elastic_thickness=100)

Plot the estimated flexure:

In [ ]:
plt.figure(figsize=(8, 4))
plt.title("Predicted flexure in Oahu")
# Fill in with your own code


plt.show()

Now try to vary $T_e$ to fit the data:
* Which part of the data should we try to fit?
* Use common sense to experiment with values. 

## Questions

Discuss the following questions with your classmates:

* What approximations were made in these calculations (physical, geophysical, and mathematical)?
* Which factors contribute to the bias and uncertainty of your $T_e$ estimate?
* How could this estimate be improved?